In [1]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import csv

class AudioAugmentation():
    # ----------------------------
    # Load an audio file. Return the signal as a tensor and the sample rate
    # ----------------------------
    @staticmethod
    def open(fullPath):
        signal, samplingRate = torchaudio.load(fullPath)
        return (signal, samplingRate)
    # ----------------------------
    # Trim signal from front
    # ----------------------------
    @staticmethod
    def timeShiftTrimFront(signal, shiftAmount):
        return signal[: , shiftAmount:]
    # ----------------------------
    # Trim signal from back
    # ----------------------------
    @staticmethod
    def timeShiftTrimBack(signal, shiftAmount):  
        return signal[: , :-shiftAmount]
    # ----------------------------
    # Generates time shift up to 30%
    # ----------------------------
    @staticmethod
    def timeShiftAmountGen(signalLength, shiftLimit):
        return int(random.random() * shiftLimit * signalLength)
    # ----------------------------
    # Updates CSV file
    # ----------------------------
    @staticmethod
    def updateCSV(classSerial, className, sampleName):
        with open('audioSampleFiles/metadata.csv', mode='a') as metadata:
            csvWriter = csv.writer(metadata, delimiter=',', quoting=csv.QUOTE_MINIMAL)
            csvWriter.writerow([classSerial, className, sampleName])
    # ----------------------------
    # Saves file, Updates CSV
    # ----------------------------
    @staticmethod
    def saveAudio(audio, samplingRate, sampleName, className, classSerial, label):
        currentPath = Path.cwd()
        sampleName = sampleName.replace(".wav", "")
        sampleName = sampleName + label + ".wav"
        outputPath = str(currentPath) + "/audioSampleFiles/readySamples/" + className + "/" + sampleName
        torchaudio.save(outputPath, audio, samplingRate, format="wav")
        AudioAugmentation.updateCSV(classSerial, className, sampleName)
    # ----------------------------
    # Stitching the paddings with the trimmed signals
    # ----------------------------
    @staticmethod
    def timeShiftStitching(signal, samplingRate, shiftAmount, sampleName, className, classSerial, label): 
        numRows, signalLength = signal.shape
        #randomly distribute padding across start and end
        paddingBefore = random.randint(0, shiftAmount)
        paddingAfter = shiftAmount - paddingBefore
        #create padding of zeros
        paddingBefore = torch.zeros((numRows, paddingBefore))
        paddingAfter = torch.zeros((numRows, paddingAfter))
        #attach zero paddings           
        shifted = torch.cat((paddingBefore, signal, paddingAfter), 1)
        #save shifted and update csv
        AudioAugmentation.saveAudio(shifted, samplingRate, sampleName, className, classSerial, label)

    # ----------------------------
    # Creates a 2 timeshifts per sample up to 30% (random amount), trimming one from front and one from back
    # Padding is distributed randomly before and affter the sample
    # ----------------------------
    @staticmethod
    def timeShiftComplete(audio, sampleName, className, classSerial):
        shiftLimit = 0.3
        signal, samplingRate = audio
        numRows, signalLength = signal.shape
        shiftAmount = AudioAugmentation.timeShiftAmountGen(signalLength, shiftLimit)
        frontTrim = AudioAugmentation.timeShiftTrimFront(signal, shiftAmount)
        backTrim = AudioAugmentation.timeShiftTrimBack(signal, shiftAmount)
        AudioAugmentation.timeShiftStitching(frontTrim, samplingRate, shiftAmount, sampleName, className, classSerial, "F")
        AudioAugmentation.timeShiftStitching(backTrim, samplingRate, shiftAmount, sampleName, className, classSerial, "B")

In [2]:
# ----------------------------
# Process the audio files in a given foder (type)
# ----------------------------
def augmentSamplesInType(preprocessedFolder, className, classSerial, samplingRate):
    typeFolder = str(preprocessedFolder) + "/" + className
    sampleNames = os.listdir(typeFolder)
    for sampleName in sampleNames:
        fullPath = str(typeFolder) + "/" + sampleName
        audio= AudioAugmentation.open(fullPath)
        AudioAugmentation.timeShiftComplete(audio, sampleName, className, classSerial)

In [3]:
# ----------------------------
# Moves the preprocessedSamples into readySamples so that the next round of Augmentation will not includee them
# ----------------------------
import shutil
def archive():
    inputFolder = str(Path.cwd()/'audioSampleFiles/preprocessedSamples')
    outputFolder = str(Path.cwd()/'audioSampleFiles/readySamples')
    classes = os.listdir(inputFolder)
    for currentClass in classes:
        srcFolder = inputFolder + "/" + currentClass
        destFolder = outputFolder + "/" + currentClass
        files = os.listdir(srcFolder)
        for file in files:
            srcPath = srcFolder + "/" + file
            destPath = destFolder + "/" + file
            shutil.move(srcPath, destPath)

In [4]:
import pandas as pd
from pathlib import Path
import os
import numpy as np

samplingRate = 16000
preprocessedFolder = Path.cwd()/'audioSampleFiles/preprocessedSamples'
classes = os.listdir(preprocessedFolder)

for currentClass in classes:
    if str(currentClass) == "lockpicking":
        augmentSamplesInType(preprocessedFolder, currentClass, 0, samplingRate)
    elif str(currentClass) == "unlocking":      
        augmentSamplesInType(preprocessedFolder, currentClass, 1, samplingRate)
    else:
        augmentSamplesInType(preprocessedFolder, currentClass, 2, samplingRate)
archive()